In [1]:
#import libraries
import xgboost as xg
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib as plt
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization, LeakyReLU
from sklearn.metrics import r2_score
import data_ML as ML
import pickle
from keras.models import load_model

np.set_printoptions(suppress=True)

In [2]:
df = ML.getdata(True)

def modifydata(df):
    data_calls = []
    data_puts = []
    df_puts = df[df.optionType == 0].drop(columns='optionType')
    df_calls = df[df.optionType == 1].drop(columns='optionType')
    df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
    df_american_puts = df_puts[df_puts.method == 1].drop(columns='method')
    df_american_calls = df_american_calls.drop(columns='index')
    df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
    df_american_puts = df_american_puts[df_american_puts.lastPrice > 5]
    df_american_puts = df_american_puts.drop(columns='index')
    y_calls = np.asarray(df_american_calls[['lastPrice']])
    y_puts = np.asarray(df_american_puts[['lastPrice']])
    df_american_calls = df_american_calls.drop(columns='lastPrice')
    df_american_puts = df_american_puts.drop(columns='lastPrice')

    for line in range(len(df_american_calls)):
        data_tmp = df_american_calls.iloc[line]
        data_calls.append(data_tmp)
    data_calls = np.asarray(data_calls)


    for line in range(len(df_american_puts)):
        data_tmp = df_american_puts.iloc[line]
        data_puts.append(data_tmp)
    data_puts = np.asarray(data_puts)

    return data_calls, data_puts, y_calls, y_puts

In [3]:
X_calls, X_puts, y_calls, y_puts = modifydata(df)
X_train_calls, X_test_calls, y_train_calls, y_test_calls = train_test_split(X_calls, y_calls, test_size=0.25)
X_train_puts, X_test_puts, y_train_puts, y_test_puts = train_test_split(X_puts, y_puts, test_size=0.25)

In [4]:
print(X_test_calls.shape)
print(X_train_calls.shape)
print(X_test_puts.shape)
print(X_train_puts.shape)

(9063, 5)
(27187, 5)
(6065, 5)
(18194, 5)


In [6]:
# # Define the model
# model_american_puts = Sequential()
# model_american_puts.add(Dense(512, input_dim=5, activation='relu'))
# model_american_puts.add(Dropout(0.0))
# model_american_puts.add(Dense(256, activation='relu'))
# model_american_puts.add(Dropout(0.0))
# model_american_puts.add(Dense(256, activation='relu'))
# model_american_puts.add(Dropout(0.0))
# model_american_puts.add(Dense(128, activation='relu'))
# model_american_puts.add(Dense(1))
# model_american_puts.compile(optimizer='adam', loss='mse')


# Define the model
model_american_puts = Sequential()
model_american_puts.add(Dense(256, input_dim=5))
model_american_puts.add(LeakyReLU(alpha=lr))
model_american_puts.add(Dropout(dr))
model_american_puts.add(Dense(128))
model_american_puts.add(LeakyReLU(alpha=lr))
model_american_puts.add(Dropout(dr))
model_american_puts.add(Dense(64))
model_american_puts.add(LeakyReLU(alpha=lr))
model_american_puts.add(Dropout(dr))
model_american_puts.add(Dense(64))
model_american_puts.add(LeakyReLU(alpha=lr))
model_american_puts.add(Dropout(dr))
model_american_puts.add(Dense(1))


# Compile the model
model_american_puts.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
# Train the model on the training set
model_american_puts.fit(X_train_puts, y_train_puts, epochs=200, batch_size=128)

# Evaluate the model on the testing set
loss, accuracy = model_american_puts.evaluate(X_test_puts, y_test_puts)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
# history = model_american_puts.fit(X_train_puts, y_train_puts, epochs=50, validation_split=0.15, batch_size=200)

Epoch 1/200
143/143 [==============================] - 1s 2ms/step - loss: 43788.1680 - accuracy: 0.0000e+00
Epoch 2/200
143/143 [==============================] - 0s 3ms/step - loss: 36132.1172 - accuracy: 0.0000e+00
Epoch 3/200
143/143 [==============================] - 0s 2ms/step - loss: 35302.9102 - accuracy: 0.0000e+00
Epoch 4/200
143/143 [==============================] - 0s 2ms/step - loss: 32425.1211 - accuracy: 0.0000e+00
Epoch 5/200
143/143 [==============================] - 0s 2ms/step - loss: 29830.9688 - accuracy: 0.0000e+00
Epoch 6/200
143/143 [==============================] - 0s 2ms/step - loss: 25169.4082 - accuracy: 0.0000e+00
Epoch 7/200
143/143 [==============================] - 0s 2ms/step - loss: 20470.7441 - accuracy: 0.0000e+00
Epoch 8/200
143/143 [==============================] - 0s 2ms/step - loss: 16434.6094 - accuracy: 0.0000e+00
Epoch 9/200
143/143 [==============================] - 0s 2ms/step - loss: 14079.8643 - accuracy: 0.0000e+00
Epoch 10/200
143/14

KeyboardInterrupt: 

In [ ]:
#prediction
import pandas as pd
y_pred = model_american_puts.predict(X_test_puts)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test_puts.tolist()})
error = (y_pred - y_test_puts)/y_test_puts
db = db.assign(error = error)
rmse = np.sqrt(mean_squared_error(y_test_puts, y_pred))
print(rmse)
db.to_csv('./Data/MLPdata.csv')
print(db)

129/129 [==============================] - 0s 412us/step
69.87894820304307
                prediction       real     error
0           [1068.4609375]  [1123.95] -0.049370
1      [136.2888641357422]   [208.34] -0.345834
2      [53.64029312133789]    [89.06] -0.397706
3      [82.48561096191406]    [64.52]  0.278450
4      [7.267213344573975]      [7.4] -0.017944
...                    ...        ...       ...
4109  [28.596813201904297]    [44.31] -0.354619
4110   [6.906627178192139]     [5.15]  0.341093
4111     [72.845458984375]   [114.14] -0.361789
4112  [196.70640563964844]    [160.0]  0.229415
4113   [9.265668869018555]     [13.0] -0.287256

[4114 rows x 3 columns]


In [7]:
# Define the model
model_american_calls = Sequential()
model_american_calls.add(Dense(512, input_dim=5, activation='relu'))
model_american_calls.add(Dropout(0.0))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.0))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.0))
model_american_calls.add(Dense(128, activation='relu'))
model_american_calls.add(Dense(1))
model_american_calls.compile(optimizer='adam', loss='mse')

history = model_american_calls.fit(X_train_calls, y_train_calls, epochs=40, validation_split=0.15, batch_size=180)

Epoch 1/40
129/129 [==============================] - 1s 3ms/step - loss: 33273.7812 - val_loss: 8562.5410
Epoch 2/40
129/129 [==============================] - 0s 3ms/step - loss: 7211.9688 - val_loss: 3808.3118
Epoch 3/40
129/129 [==============================] - 0s 3ms/step - loss: 4477.4497 - val_loss: 2816.4504
Epoch 4/40
129/129 [==============================] - 0s 3ms/step - loss: 5289.2388 - val_loss: 2644.4373
Epoch 5/40
129/129 [==============================] - 0s 3ms/step - loss: 3103.8958 - val_loss: 1072.6077
Epoch 6/40
129/129 [==============================] - 0s 3ms/step - loss: 3069.1709 - val_loss: 4352.2021
Epoch 7/40
129/129 [==============================] - 0s 3ms/step - loss: 2816.3420 - val_loss: 980.8353
Epoch 8/40
129/129 [==============================] - 0s 3ms/step - loss: 2295.9700 - val_loss: 1944.4431
Epoch 9/40
129/129 [==============================] - 0s 3ms/step - loss: 2085.2898 - val_loss: 2266.8276
Epoch 10/40
129/129 [=========================

In [8]:
#prediction
import pandas as pd
y_pred = model_american_calls.predict(X_test_calls)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test_calls.tolist()})
error = (y_pred - y_test_calls)/y_test_calls
db = db.assign(error = error)
rmse = np.sqrt(mean_squared_error(y_test_calls, y_pred))
print(rmse)
db.to_csv('./Data/MLPdata.csv')
db


284/284 [==============================] - 0s 618us/step
46.39956433111765


,prediction,real,error
0,[4.437221527099609],[6.46],-0.313124
1,[43.325958251953125],[42.45],0.020635
2,[109.45326232910156],[111.95],-0.022302
3,[65.62226104736328],[67.94],-0.034114
4,[33.524166107177734],[27.63],0.213325
...,...,...,...
9058,[11.995805740356445],[18.62],-0.355757
9059,[435.64874267578125],[469.55],-0.072199
9060,[330.36981201171875],[321.4],0.027909
9061,[124.14932250976562],[116.4],0.066575


In [ ]:
model_american_calls.save("model_american_calls.h5")
model_american_puts.save("model_american_puts.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# XGBr for American call options
XGBr = xg.XGBRegressor(learning_rate=0.1, n_estimators=900)
cvs_XGBr = cross_val_score(XGBr, X_train_calls, y_train_calls, scoring='neg_root_mean_squared_error', cv=KFold(n_splits=10))
XGBr_score = cvs_XGBr.mean()
print(XGBr_score)

-5.667180249194848


In [10]:
XGBr.fit(X_train_calls, y_train_calls)
XGBr_pred = XGBr.predict(X_test_calls)
XGBr_rmse = np.sqrt(mean_squared_error(y_test_calls, XGBr_pred))
XGBr_rmse

5.578737616638425

In [ ]:
# Save the trained model to a file
filename = 'XGBr_model_calls.sav'
pickle.dump(XGBr, open(filename, 'wb'))

In [11]:
y_pred = pd.DataFrame(XGBr_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test_calls, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,6.46,6.810596
1,42.45,42.794170
2,111.95,112.253769
3,67.94,67.699951
4,27.63,26.572945
...,...,...
9058,18.62,18.840527
9059,469.55,506.471710
9060,321.40,321.501312
9061,116.40,117.397720


In [12]:
# XGBr for American call options
XGBr = xg.XGBRegressor(learning_rate=0.1, n_estimators=900)
cvs_XGBr = cross_val_score(XGBr, X_train_puts, y_train_puts, scoring='neg_root_mean_squared_error', cv=KFold(n_splits=10))
XGBr_score = cvs_XGBr.mean()
print(XGBr_score)

-8.965576499724211


In [13]:
XGBr.fit(X_train_puts, y_train_puts)
XGBr_pred = XGBr.predict(X_test_puts)
XGBr_rmse = np.sqrt(mean_squared_error(y_test_puts, XGBr_pred))
XGBr_rmse

11.061345948392876

In [ ]:
# Save the trained model to a file
filename = 'XGBr_model_puts.sav'
pickle.dump(XGBr, open(filename, 'wb'))

In [14]:
y_pred = pd.DataFrame(XGBr_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test_puts, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,283.68,284.167297
1,125.32,124.984299
2,6.25,6.190952
3,43.00,43.269341
4,482.70,482.800018
...,...,...
6060,71.10,69.383018
6061,22.30,22.238087
6062,15.90,15.990765
6063,13.05,12.976566


In [50]:
DTR_calls = DecisionTreeRegressor()
# cvs_DTR = cross_val_score(DTR, df_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# DTR_score = cvs_DTR.mean()
# print(DTR_score)

DTR_calls.fit(X_train_calls, y_train_calls)
DTR1_pred = DTR_calls.predict(X_test_calls)
DTR1_rmse = np.sqrt(mean_squared_error(y_test_calls, DTR1_pred))
DTR1_rmse

2.6290923366057735

In [51]:
y_pred = pd.DataFrame(DTR1_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test_calls, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,6.46,6.46
1,42.45,42.45
2,111.95,111.95
3,67.94,67.94
4,27.63,26.15
...,...,...
9058,18.62,18.62
9059,469.55,469.55
9060,321.40,321.40
9061,116.40,116.40


In [52]:
# Save the trained model to a file
filename = 'DTR_model_calls.sav'
pickle.dump(DTR_calls, open(filename, 'wb'))

In [45]:
DTR_puts = DecisionTreeRegressor()
# cvs_DTR = cross_val_score(DTR, df_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# DTR_score = cvs_DTR.mean()
# print(DTR_score)

DTR_puts.fit(X_train_puts, y_train_puts)
DTR1_pred = DTR_puts.predict(X_test_puts)
DTR1_rmse = np.sqrt(mean_squared_error(y_test_puts, DTR1_pred))
DTR1_rmse

5.4102475314162275

In [47]:
y_pred = pd.DataFrame(DTR1_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test_puts, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,283.68,283.68
1,125.32,120.67
2,6.25,6.25
3,43.00,43.00
4,482.70,482.70
...,...,...
6060,71.10,71.10
6061,22.30,22.30
6062,15.90,15.90
6063,13.05,13.05


In [48]:
# Save the trained model to a file
filename = 'DTR_model_puts.sav'
pickle.dump(DTR_puts, open(filename, 'wb'))

In [ ]:
df = ML.getdata(True)

def modifydata(df):
    data_calls = []
    data_puts = []
    df_puts = df[df.optionType == 0].drop(columns='optionType')
    df_calls = df[df.optionType == 1].drop(columns='optionType')
    df_eu_calls = df_calls[df_calls.method == 0].drop(columns='method')
    df_eu_puts = df_puts[df_puts.method == 0].drop(columns='method')
    df_eu_calls = df_eu_calls.drop(columns='index')
    df_eu_calls = df_eu_calls[df_eu_calls.lastPrice > 5]
    df_eu_puts = df_eu_puts[df_eu_puts.lastPrice > 5]
    df_eu_puts = df_eu_puts.drop(columns='index')
    y_calls = np.asarray(df_eu_calls[['lastPrice']])
    y_puts = np.asarray(df_eu_puts[['lastPrice']])
    df_eu_calls = df_eu_calls.drop(columns='lastPrice')
    df_eu_puts = df_eu_puts.drop(columns='lastPrice')

    for line in range(len(df_eu_calls)):
        data_tmp = df_eu_calls.iloc[line]
        data_calls.append(data_tmp)
    data_calls = np.asarray(data_calls)


    for line in range(len(df_eu_puts)):
        data_tmp = df_eu_puts.iloc[line]
        data_puts.append(data_tmp)
    data_puts = np.asarray(data_puts)

    return data_calls, data_puts, y_calls, y_puts